In [1]:
import cv2
from ultralytics import YOLO
import easyocr
import os
import numpy as np

image_path = '/mnt/c/Users/Rahul/Desktop/Test/Passport/a.jpg'
model_path = "/mnt/c/Users/Rahul/Desktop/trained models/withoutdocumenttypw_Passport_model.pt"

model = YOLO(model_path)

results = model(image_path)

image = cv2.imread(image_path)

ocr = easyocr.Reader(['ne', 'en'])

class_map = {
    0: 'citizenship_number',
    1: 'dob',
    2: 'document_type',
    3: 'name',
    4: 'passport_number',
    5: 'surname',
}

def get_color(class_id):
    np.random.seed(class_id)
    return tuple(np.random.randint(0, 255, 3).tolist())

base_name = os.path.splitext(os.path.basename(image_path))[0]

output_dir = 'OCR_Outputs'
os.makedirs(output_dir, exist_ok=True)

output_file_path = os.path.join(output_dir, f'{base_name}_results.txt')

with open(output_file_path, 'w') as file:
    for result in results:
        boxes = result.boxes.xyxy
        class_ids = result.boxes.cls

        for box, class_id in zip(boxes, class_ids):
            x1, y1, x2, y2 = map(int, box.tolist())
            
            cropped_img = image[y1:y2, x1:x2]

            ocr_result = ocr.readtext(cropped_img)

            for detection in ocr_result:
                text = detection[1]
                file.write(f"Class: {class_map.get(int(class_id), 'Unknown')}, Text: {text}\n")
                    
            color = get_color(int(class_id))
            label = class_map.get(int(class_id), 'Unknown')
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

annotated_image_path = os.path.join(output_dir, f'{base_name}_annotated.jpg')
cv2.imwrite(annotated_image_path, image)



image 1/1 /mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/notebooks/YOLOv8/runs/detect/Passport/weights/hajurba.JPG: 800x608 1 citizenship_number, 2 dobs, 2 document_types, 1 name, 2 passport_numbers, 2 surnames, 67.3ms
Speed: 4.9ms preprocess, 67.3ms inference, 250.4ms postprocess per image at shape (1, 3, 800, 608)


True